# Compatible embeddings using Allombert's algorithm

## Code

We start by defining some basic classes to represent 

1. triple $(K, \chi, \alpha)$, where $K\cong\mathbb F_{p^n}$ is a finite field, $\chi$ is the minimal polynomial over $k=\mathbb F_p$ of a $n$-th primitive root of unity $\zeta$, and $\alpha$ is a solution of Hilbert 90 in the algebra $K\otimes k(\zeta)$.
2. a couple $(c, f)$, where $f:K_1\hookrightarrow K_2$ is an embedding between two finite fields $K_1$ and $K_2$ and $c$ is the constant used in Allombert's algorithm to define the embedding.

In [1]:
class ffembed:
    
    def __init__(self, k):
        self._field = k
        self._root = None
        self._hilbert_gen = None
    def __repr__(self):
        return self._field.__repr__()
    
    def has_root(self):
        return self._root != None
    
    def root(self):
        return self._root
    
    def hilbert_gen(self):
        return self._hilbert_gen
    
    def field(self):
        return self._field
    
    def assign_root(self, r, g):
        self._root = r
        self._hilbert_gen = g

class embedding:
    
    def __init__(self, c, f):
        self._constant = c
        self._map = f
        
    def __call__(self, x):
        return self._map(x)
    
    def constant(self):
        return self._constant
    
    def get_map(self):
        return self._map

We define functions to solve Hilbert 90.

In [2]:
"""
Compute the frobenius σ' of `elem` in K⊗C defined by σ'(x⊗y) = σ(x)⊗y with 
σ the frobenius of K/k with k the prime subfield.
"""
def froby(elem):
    l = elem.list()
    f = parent(l[0]).frobenius_endomorphism()
    res = parent(elem)()
    t = parent(elem).gen()
    for j in range(len(l)):
        res += f(l[j])*t^j
    return res  

"""
Compute the matrix representing the frobenius σ⊗Id in K⊗C, over the basis B⊗1
where B is a basis of K over k. The coefficients are in C=k(ζ).
"""
def frob_mat(K, C = None):
    
    if C == None:
        C = K.prime_subfield()
    
    frob = K.frobenius_endomorphism()
    n = K.degree()
    S = MatrixSpace(C, n)
    g = K.gen()
    
    cols = []
    
    for j in range(n):
        s = frob(g^j)
        l = s.polynomial().list()
        l = l + (n-len(l))*[C()]
        cols.append(l)
        
    return S(cols).transpose()

"""
Compute a solution of the equation ``σ'(x) = (1⊗ζ)x`` in K⊗C = T.
"""
def hilbert90(K, C, T):
    
    M = frob_mat(K, C)
    t = C.gen()
    eigen = M.right_eigenvectors()
    
    for e in eigen:
        if e[0] == t:
            v = e[1][0]
            break
    
    res = T()
    g = K.gen()
    for j in range(K.degree()):
        res += T(v[j])*g^j
        
    return res

We define functions to compute compatible roots of unity.

In [3]:
"""
Compute the first coordinate of `elem` in `basis`, knowing that
`elem` is in fact in a subspace of degree `d2`.
"""
def change_basis(elem, basis, d2):
    
    k = elem.base_ring()
    d1 = elem.parent().degree()
    A = MatrixSpace(k, d1, d2)()
    
    for i in range(d2):
        L = (basis^i).list()
        for j in range(d1):
            A[j,i] = L[j]
            
    S2 = MatrixSpace(k, d1, 1)
    B = S2(elem.list())
    X = A.solve_right(B)
    
    return X[0,0]

"""
Compute a n-th primitive root of unity, where n is equal to the degree of `K1`. 
We do not impose any condition on that root.
"""
def compute_root_no_cond(K1):
    
    k1 = K1.field()
    m = k1.degree()
    k = k1.prime_subfield()
    R = PolynomialRing(k, "x")
    f1 = factor(R(cyclotomic_polynomial(m)))[0][0]
    C1 = k.extension(f1, "t")
    T1 = k1.extension(f1, "t")
    a1 = hilbert90(k1, C1, T1)
    
    K1.assign_root(f1, a1)

"""
Compute a n-th primitive root of unity, where n is equal to the degree of `K2`.
The root must be compatible with the root in `K1`, a smaller field of degree m.
This means we want that ζ2^(n/m) = ζ1, where ζi is the root chosen for Ki.
"""
def compute_root_compatible1(K2, K1):
    
    k1, k2 = K1.field(), K2.field()
    m, n = k1.degree(), k2.degree()
    k = k1.prime_subfield()
    R = PolynomialRing(k, "x")
    Rxy.<x,y> = PolynomialRing(k, "x, y")
    f1 = K1.root()
    f1xy = Rxy(f1)
    res = f1xy.resultant(y^(n/m)-x, x)
    P = gcd(R(res.polynomial(y).list()), R(cyclotomic_polynomial(n)))
    f2 = factor(P)[0][0]
    C2 = k.extension(f2, "t")
    T2 = k2.extension(f2, "t")
    a2 = hilbert90(k2, C2, T2)
    
    K2.assign_root(f2, a2)
        
"""
Compute a m-th primitive root of unity, where m is equal to the degree of `K1`.
The root must be compatible with the root in `K2`, a bigger field of degree n.
This means we want that ζ2^(n/m) = ζ1, where ζi is the root chosen for Ki.
"""
def compute_root_compatible2(K1, K2):
    
    k1, k2 = K1.field(), K2.field()
    m, n = k1.degree(), k2.degree()
    k = k1.prime_subfield()
    R = PolynomialRing(k, "x")
    Rxy.<x,y> = PolynomialRing(k, "x, y")
    f2 = K2.root()
    f2xy = Rxy(f2)
    res = f2xy.resultant(x^(n/m)-y, x)
    P = gcd(R(res.polynomial(y).list()), R(cyclotomic_polynomial(m)))
    f1 = factor(P)[0][0]
    C1 = k.extension(f1, "t")
    T1 = k1.extension(f1, "t")
    a1 = hilbert90(k1, C1, T1)
    
    K1.assign_root(f1, a1)

"""
Compute a n-th primitive root of unity, where n is equal to the degree of `K2`.
If L = [], we do not impose any compatibility condition, otherwise we impose the conditions 
relative to the roots of the fields in L.
"""
def compute_root(K2, L = []):
    
    if type(L) != list:
        L = [L]
    
    if L == []:
        compute_root_no_cond(K2)
        
    elif len(L) == 1:
        if K2.field().degree() >= L[0].field().degree():
            compute_root_compatible1(K2, L[0])
        else:
            compute_root_compatible2(K2, L[0])
                
    else:
        
        k2 = K2.field()
        n = k2.degree()
        k = k2.prime_subfield()
        R = PolynomialRing(k, "x")
        Rxy.<x,y> = PolynomialRing(k, "x, y")
        P = R(cyclotomic_polynomial(n))
        
        for K1 in L:

            k1 = K1.field()
            m =  k1.degree()
            f1 = K1.root()
            f1xy = Rxy(f1)
            res = f1xy.resultant(y^(n/m)-x, x)
            P = gcd(R(res.polynomial(y).list()), P)
            
        f2 = factor(P)[0][0]    
        C2 = k.extension(f2, "t")
        T2 = k2.extension(f2, "t")
        a2 = hilbert90(k2, C2, T2)

        K2.assign_root(f2, a2)


Finally we define functions to compute embeddings.

In [4]:
"""
Compute the matrix whose columns are the powers of a form a^0 to a^(n-1).
"""
def basis_matrix(a, n = None):
    K = a.parent()
    m = K.degree()
    
    if n == None:
        n = m
    
    k = K.prime_subfield()
    S = MatrixSpace(k, m, n)()
    for j in range(n):
        L = (a^j).polynomial().list()
        i = 0
        for l in L:
            S[i, j] = l
            i += 1
    return S

def fflist(x):
    L = x.polynomial().list()
    l = len(L)
    k = x.parent()
    d = k.degree()
    if l < d:
        L += (d-l)*[k()]
        
    return L

"""
Compute the linear map φ: a|-> b sending a to b.
"""
def compute_map(a, b):
    
    A = basis_matrix(a)
    B = basis_matrix(b, a.parent().degree())
    C = B*A^(-1)
    
    K = b.parent()
    k = K.prime_subfield()
    S = MatrixSpace(k, a.parent().degree(), 1)

    return lambda x : K((C*S(fflist(x))).column(0))

"""
Compute an embedding from K1 to K2.
"""
def compute_embedding_no_cond(K1, K2):
    
    a1 = K1.hilbert_gen()
    a2 = K2.hilbert_gen()
    k1, k2 = K1.field(), K2.field()
    k = k1.prime_subfield()
    m, n = k1.degree(), k2.degree()
    C1 = k.extension(K1.root(), "t")
    C2 = k.extension(K2.root(), "t")
    T2 = a2.parent()
       
    b1 = C2([k(x) for x in (a1^m).list()]).polynomial().subs(C2.gen()^(n/m))
    b2 = C2([k(x) for x in (a2^n).list()])
    c = (b1*b2^(-1)).nth_root(m)
    
    a, b = a1.list()[0], change_basis(T2(c)*a2^(n/m), T2.gen()^(n/m), C1.degree())
    f = compute_map(a, b)
    return embedding(c, f)

"""
Compute an embedding from K1 to K2. The resulting embedding is compatible with 
the existing morphism f01 from K0 to K1 and f02 from K0 to K2.
"""
def compute_embedding_compatible(K1, K2, f01, f02, K0):
    
    a1 = K1.hilbert_gen()
    a2 = K2.hilbert_gen()
    k1, k2 = K1.field(), K2.field()
    k = k1.prime_subfield()
    l, m, n = K0.field().degree(), k1.degree(), k2.degree()
    C1 = k.extension(K1.root(), "t")
    C2 = k.extension(K2.root(), "t")
    T2 = a2.parent()
       
    b1 = C2([k(x) for x in (a1^m).list()]).polynomial().subs(C2.gen()^(n/m))
    b2 = C2([k(x) for x in (a2^n).list()])
    e = (b1*b2^(-1)).nth_root(m)
    c, d = f01.constant().polynomial().subs(C2.gen()^(n/m)), f02.constant()
    
    zeta = (e^(m/l)*c*d^(-1)).nth_root(m/l)
    e = e*zeta^(-1)
    
    a, b = a1.list()[0], change_basis(T2(e)*a2^(n/m), T2.gen()^(n/m), C1.degree())
    f = compute_map(a, b)
    return embedding(e, f)

## Examples

### Triangle

In [5]:
p = 5
k = GF(p)
k12 = GF(p^12)
k24 = GF(p^24)
k48 = GF(p^48)

In [6]:
K12 = ffembed(k12)
K24 = ffembed(k24)
K48 = ffembed(k48)

In [7]:
compute_root(K12)
compute_root(K48, K12)
f = compute_embedding_no_cond(K12, K48)
compute_root(K24, K48)
g = compute_embedding_no_cond(K12, K24)
h = compute_embedding_compatible(K24, K48, g, f, K12)
hh = compute_embedding_no_cond(K24, K48)

In [8]:
z = k12.gen(); z

z12

In [9]:
f(z) == h(g(z))

True

In [10]:
f(z) == hh(g(z))

False

### Diamond

In [11]:
k7 = GF(p^7)
k21 = GF(p^21)
k14 = GF(p^14)
k42 = GF(p^42)

K7 = ffembed(k7)
K14 = ffembed(k14)
K21 = ffembed(k21)
K42 = ffembed(k42)

In [12]:
compute_root(K14)
compute_root(K21)
compute_root(K42, [K14, K21])

In [13]:
C.<t> = k.extension(K42.root(), "t")

In [14]:
(t^2).minpoly() == K21.root()

True

In [15]:
(t^3).minpoly() == K14.root()

True

In [16]:
compute_root(K7, K14)

In [17]:
(t^6).minpoly() == K7.root()

True

In [18]:
f = compute_embedding_no_cond(K7, K42)
g = compute_embedding_no_cond(K7, K21)
h = compute_embedding_compatible(K21, K42, g, f, K7)
i = compute_embedding_no_cond(K7, K14)
j = compute_embedding_compatible(K14, K42, i, f, K7)

In [19]:
x = k7.gen()

In [20]:
f(x) == h(g(x))

True

In [21]:
j(i(x)) == h(g(x))

True

In [22]:
f.constant()

3*t^5 + 2*t^4 + 3*t^3 + 3*t^2 + 4*t + 3

# Tests

In [14]:
k = GF(3)
r = PolynomialRing(k, "t")
p = r.gen()^2-2

In [24]:
k2.<t> = k.extension(p)
R = PolynomialRing(k2, "w")

In [27]:
P = R.gen()^2-(t+1)
factor(P)

w^2 + 2*t + 2

In [26]:
P.is_irreducible()

True

In [28]:
k4.<w> = k2.extension(P)

In [30]:
k4.is_field()

True

In [34]:
(-t)^2

2

In [40]:
(t-1)^3 == t*(t-1)

True

In [41]:
t.minpoly()

x^2 + 1

In [42]:
(t-1).minpoly()

x^2 + 2*x + 2

In [47]:
k4

Univariate Quotient Polynomial Ring in w over Finite Field in t of size 3^2 with modulus w^2 + 2*t + 2

In [48]:
R4.<T> = PolynomialRing(k4, "T")

In [52]:
P = (T^3-T)^3 + (T^3-T) -2*T

In [54]:
P

T^9

# Understanding something

About a tower of extension

or

# Case $p=3$ and $\ell = 2$

In [2]:
p = 3
k = GF(p)
k4 = GF(p^4)
R = PolynomialRing(k, "x")
R4 = PolynomialRing(k4, "x")
z4 = k4.gen()

In [3]:
C = k.extension(R(cyclotomic_polynomial(4)), "t")

In [7]:
T = k4.extension(R4(cyclotomic_polynomial(4)), "t")

In [8]:
factor(R4(cyclotomic_polynomial(4)))

(x + z4^3 + z4^2 + 1) * (x + 2*z4^3 + 2*z4^2 + 2)

In [9]:
zeta1 = -(z4^3 + z4^2 + 1)

In [10]:
zeta2 = -(2*z4^3 + 2*z4^2 + 2)

In [11]:
M = frob_mat(k4, C)
t = C.gen()
eigen = M.right_eigenvectors()

In [20]:
eigen[3][0]

t

In [25]:
zeta1 + zeta2 == 0

True

In [22]:
eigen

[(2, [
  (1, 0, 1, 1)
  ], 1), (1, [
  (1, 0, 0, 0)
  ], 1), (2*t, [
  (1, 2*t, 2*t + 2, 2*t)
  ], 1), (t, [
  (1, t, t + 2, t)
  ], 1)]

In [24]:
(2*t).minpoly()

x^2 + 1

In [7]:
def hilbert91(K, C, T):
    
    M = frob_mat(K, C)
    t = C.gen()
    eigen = M.right_eigenvectors()
    p = K.characteristic()
    
    N = C.modulus().degree()
    L = []
    zetas = [t^(p^j) for j in range(N)]
    
    for e in eigen:
        v = e[1][0]
        res = T()
        g = K.gen()
        for j in range(K.degree()):
            res += T(v[j])*g^j

        L.append((e[0], res))
    return L

In [12]:
L = hilbert91(k4, C, T); L

[(2, z4^3 + z4^2 + 1),
 (1, 1),
 (2*t, (2*z4^3 + 2*z4^2 + 2*z4)*t + 2*z4^2 + 1),
 (t, (z4^3 + z4^2 + z4)*t + 2*z4^2 + 1)]

In [13]:
a0 = L[3][1]; a1 = L[2][1]; a0, a1

((z4^3 + z4^2 + z4)*t + 2*z4^2 + 1, (2*z4^3 + 2*z4^2 + 2*z4)*t + 2*z4^2 + 1)

In [14]:
a0^2

(2*z4^3 + 2*z4^2 + 2)*t + 2*z4^3 + 2*z4^2 + 2

In [15]:
(a0^2)/(a1^2)

t

In [16]:
b = L[0][1];b

z4^3 + z4^2 + 1

In [17]:
c0 = a0^2/b; c0

2*t + 2

In [18]:
c1 = a1^2/b; c1

t + 2

In [67]:
z4 = k4.gen()

In [77]:
(z4^3 + z4^2 + 1)^2

2

In [84]:
c = (2/C([k(s) for s in (a0^4).list()])); c

2*t

In [85]:
c.nth_root(2)

2*t + 2

In [74]:
(2*t)^2

2

In [75]:
t.parent()

Finite Field in t of size 3^2

In [47]:
for g in C:
    print g^2

0
t
2
2*t
1
t
2
2*t
1


In [21]:
(z4^3 + z4^2 + 1)^p == -((z4^3 + z4^2 + 1))

True

In [25]:
zeta1^2

2

In [4]:
Rxy.<x, y> = PolynomialRing(k)

In [5]:
P = x^2+1

In [6]:
P.resultant(y^2-x-1, x)

y^4 + y^2 - 1

In [7]:
X = R.gen()

In [8]:
irr = X^4+X^2-1

In [9]:
irr.is_irreducible()

True

In [10]:
Qt.<t> = PolynomialQuotientRing(R, X^2+1)

In [46]:
Qt

Univariate Quotient Polynomial Ring in t over Finite Field of size 3 with modulus x^2 + 1

In [47]:
Rt.<T> = Qt[]

In [50]:
Qtt.<u> = PolynomialQuotientRing(Rt, T^2-(t+1))

In [56]:
u^4+u^2-1

0

In [11]:
r = R.gen()
f4.<a> = k.extension(r^4+r^2-1)

In [12]:
f4.modulus()

x^4 + x^2 + 2

In [13]:
t = (-a^4)

In [14]:
(t-1)^3 == t*(t-1)

True

In [15]:
P = X^2+1

In [16]:
U = f4.extension(P)

In [17]:
U

Univariate Quotient Polynomial Ring in x over Finite Field in a of size 3^4 with modulus x^2 + 1

In [23]:
M = hilbert91(f4, C, U); M

[(2, 2*a^2 + 1), (1, 1), (2*t, a^3*x + a), (t, 2*a^3*x + a)]

In [24]:
a0 = M[3][1]; a1 = M[2][1]; a0,a1

(2*a^3*x + a, a^3*x + a)

In [25]:
b0 = M[0][1]; b0

2*a^2 + 1

In [26]:
a0^2/b0

x + 1

In [27]:
a1^2/b0

2*x + 1

In [28]:
y = a0.parent().gen()

In [105]:
froby(b0) == -b0

True

In [29]:
f4.modulus()

x^4 + x^2 + 2

In [31]:
a^8

2

In [34]:
-t*(t+1) == 1-t

True

In [36]:
a0^2

(2*a^2 + 1)*x + 2*a^2 + 1

In [37]:
-t

2*a^2 + 1

In [42]:
b0

2*a^2 + 1

In [51]:
a0^2

(2*a^2 + 1)*x + 2*a^2 + 1

In [53]:
-t

2*a^2 + 1

# Case $p=5$ and $\ell=2$

In [26]:
p = 5
k = GF(p)
k4 = GF(p^4)
R = PolynomialRing(k, "x")
R4 = PolynomialRing(k4, "x")
z4 = k4.gen()

In [51]:
x = R.gen()

In [28]:
q = x^2-2

In [29]:
q.is_irreducible()

True

In [30]:
kt.<t> = k.extension(q)

In [31]:
rt.<y> = kt[]

In [32]:
w = y^2-t

In [33]:
w.is_irreducible()

True

In [34]:
ku.<u> = kt.extension(w)

In [35]:
ru.<z> = ku[]

In [36]:
r = z^2-u

In [37]:
r.is_irreducible()

True

In [38]:
u.minpoly()

x^2 + 4*t

In [39]:
t.minpoly()

x^2 + 3

In [40]:
kv.<v> = ku.extension(r)

In [43]:
v.minpoly()

x^2 + 4*u

In [46]:
Rxy.<x, y> = k[]
P = x^2-2
P.resultant(y^2-x, x)

y^4 - 2

In [50]:
v^8

2

In [52]:
factor(x^8-2)

x^8 + 3

In [53]:
k8.<V> = k.extension(x^8-2)

In [54]:
k8

Finite Field in V of size 5^8

In [78]:
factor(R(cyclotomic_polynomial(64)))

(x^16 + 2) * (x^16 + 3)

In [61]:
C = k.extension(x^2-2, "t")
T = k8.extension(x^2-2, "t")

In [62]:
hilbert91(k8, C, T)

[(4, V^4),
 (3, V^6),
 (2, V^2),
 (1, 1),
 (4*t, 4*V^7*t + V^3),
 (3*t, V^5*t + V),
 (2*t, 4*V^5*t + V),
 (t, V^7*t + V^3)]

In [64]:
bar = hilbert91(k8, C, T)

In [66]:
foo = bar[-1][1]

In [67]:
foo^2

4*V^2*t

In [69]:
(2*v)^-1

4*t*u*v

In [70]:
foo

V^7*t + V^3

In [72]:
t.parent()

Finite Field in t of size 5^2

In [74]:
(-t).is_square()

False

# Composita with $p = 7, m = 2, n = 3$

In [6]:
k = GF(7)
R.<x> = k[]
k2.<u> = k.extension(x^2-3)
k3.<t> = k.extension(x^3-2)
k6.<w> = k.extension(x^6-3)
r6.<t> = k6[]

In [26]:
k(3)^3

6

In [27]:
P = x^6-3
P.is_irreducible()

True

In [34]:
(w^5)^7

5*w^5

In [19]:
factor(t^3-2)

(t + 3*w^4) * (t + 5*w^4) * (t + 6*w^4)

In [11]:
(3*w^4)^7

5*w^4

In [12]:
_^7

6*w^4

In [14]:
j = k(4)

In [18]:
for j in k:
    print j^2

0
1
4
2
2
4
1


# Some other tests

In [6]:
p = 5
n = 9
r = Zmod(n)(p).multiplicative_order()
K = GF(p^n)
C = GF(p^r)
T = K.extension(C.modulus())
R.<x> = C[]
r

6

In [26]:
factor(R(cyclotomic_polynomial(n)))

(x + z6^4 + 4*z6^3 + 4*z6^2 + 4*z6 + 3) * (x + 2*z6^4 + 3*z6^3 + 3*z6^2 + 1) * (x + 2*z6^5 + 2*z6^4 + 4*z6^3 + 2*z6 + 2) * (x + 2*z6^5 + 4*z6^4 + z6^3 + z6^2 + 2) * (x + 3*z6^5 + z6) * (x + 3*z6^5 + z6^4 + 3*z6^3 + 2*z6^2 + 3*z6 + 2)

In [28]:
L = hilbert91(K, C, T)

In [43]:
zeta = L[1][0]; zeta

4*z6^4 + z6^3 + z6^2 + z6 + 2

In [37]:
a = L[1][1]; a

(2*z9^8 + 4*z9^7 + 2*z9^6 + z9^5 + 4*z9^4 + 4*z9^3 + z9^2 + 2*z9)*x^5 + (z9^8 + z9^6 + 4*z9^4 + 2*z9^3 + z9^2 + 3*z9)*x^4 + (3*z9^8 + 3*z9^6 + 4*z9^5 + 2*z9^4 + 2*z9^3)*x^3 + (3*z9^7 + 2*z9^5 + 2*z9^3 + 2*z9)*x^2 + (4*z9^8 + 4*z9^7 + 4*z9^6 + 3*z9^5 + 4*z9^4 + 2*z9^3 + 4*z9)*x + 3*z9^7 + 2*z9^6 + z9^4 + z9^2 + 3*z9 + 1

In [44]:
t = froby(a)/a; t

4*x^4 + x^3 + x^2 + x + 2

In [47]:
t^3, t^9

(2*x^5 + x^4 + 2*x^2 + 3*x + 3, 1)

In [50]:
a.base_ring()

Finite Field in z9 of size 5^9

In [66]:
def riso(a, K, C):
    
    p = K.characteristic()
    
    n = K.degree()
    r = C.degree()
    
    g = C.gen()
    
    m = lcm(n, r)
    d = gcd(n, r)
    
    A = GF(p^m)
    alg = A^d
    res = alg()
    
    cpt = -1
    for j in a:
        cpt += 1
        y = A(g^cpt)
        for i in range(d):
            x = A(j^(p^i))
            res[i] += x*y
    
    return res

In [51]:
kk = GF(p^30)

In [53]:
V = kk^3

In [60]:
v = V.random_element(); v

(3*z30^29 + 2*z30^28 + z30^27 + z30^25 + 2*z30^23 + 4*z30^21 + z30^20 + 2*z30^19 + 3*z30^18 + 4*z30^17 + 4*z30^16 + 3*z30^15 + 3*z30^14 + 4*z30^13 + z30^12 + z30^11 + z30^10 + 2*z30^9 + 4*z30^8 + z30^7 + 2*z30^4 + 3*z30^3 + 2*z30^2 + z30 + 1, z30^28 + 2*z30^27 + 2*z30^24 + z30^23 + z30^22 + z30^21 + 3*z30^20 + 4*z30^19 + z30^18 + z30^16 + 3*z30^15 + 3*z30^14 + z30^12 + z30^10 + z30^9 + 2*z30^8 + z30^7 + 3*z30^5 + z30^4 + z30^3 + 3*z30^2 + 4*z30 + 1, 4*z30^29 + 4*z30^28 + 4*z30^27 + z30^25 + 4*z30^23 + z30^22 + 2*z30^21 + 2*z30^20 + 2*z30^19 + 2*z30^18 + 3*z30^17 + z30^16 + z30^14 + 4*z30^13 + z30^11 + 2*z30^10 + 4*z30^9 + 3*z30^8 + z30^7 + 4*z30^5 + 3*z30^4 + 3*z30^3 + z30^2 + 3*z30 + 2)

In [61]:
v.pairwise_product(V.an_element())

(3*z30^29 + 2*z30^28 + z30^27 + z30^25 + 2*z30^23 + 4*z30^21 + z30^20 + 2*z30^19 + 3*z30^18 + 4*z30^17 + 4*z30^16 + 3*z30^15 + 3*z30^14 + 4*z30^13 + z30^12 + z30^11 + z30^10 + 2*z30^9 + 4*z30^8 + z30^7 + 2*z30^4 + 3*z30^3 + 2*z30^2 + z30 + 1, 0, 0)

In [62]:
a

(2*z9^8 + 4*z9^7 + 2*z9^6 + z9^5 + 4*z9^4 + 4*z9^3 + z9^2 + 2*z9)*x^5 + (z9^8 + z9^6 + 4*z9^4 + 2*z9^3 + z9^2 + 3*z9)*x^4 + (3*z9^8 + 3*z9^6 + 4*z9^5 + 2*z9^4 + 2*z9^3)*x^3 + (3*z9^7 + 2*z9^5 + 2*z9^3 + 2*z9)*x^2 + (4*z9^8 + 4*z9^7 + 4*z9^6 + 3*z9^5 + 4*z9^4 + 2*z9^3 + 4*z9)*x + 3*z9^7 + 2*z9^6 + z9^4 + z9^2 + 3*z9 + 1

In [64]:
kk^1

Vector space of dimension 1 over Finite Field in z30 of size 5^30

In [68]:
v[3]

IndexError: vector index out of range

In [78]:
b = riso(a, K, C); b

(4*z18^17 + 2*z18^16 + 4*z18^15 + 2*z18^14 + 2*z18^13 + 2*z18^12 + 2*z18^10 + 2*z18^9 + 2*z18^8 + 4*z18^7 + z18^6 + 2*z18^5 + 4*z18^4 + 3*z18^3 + 3*z18^2 + 3*z18, z18^17 + z18^16 + 4*z18^15 + 2*z18^14 + 4*z18^13 + z18^12 + 2*z18^10 + 3*z18^9 + z18^8 + 4*z18^7 + 3*z18^5 + z18^3 + z18^2 + z18 + 3, 3*z18^17 + 4*z18^16 + z18^15 + 2*z18^13 + 4*z18^12 + z18^11 + z18^8 + 2*z18^7 + 2*z18^5 + z18^4 + 3*z18^3 + 4*z18^2 + 4*z18)

In [79]:
l = [c^n for c in b]; l

[z18^17 + z18^16 + 3*z18^15 + 3*z18^14 + z18^13 + 2*z18^12 + 2*z18^11 + 2*z18^10 + 2*z18^9 + 4*z18^8 + 2*z18^7 + 3*z18^5 + 3*z18^4 + 4*z18 + 4,
 z18^17 + z18^16 + 3*z18^15 + 3*z18^14 + z18^13 + 2*z18^12 + 2*z18^11 + 2*z18^10 + 2*z18^9 + 4*z18^8 + 2*z18^7 + 3*z18^5 + 3*z18^4 + 4*z18 + 4,
 z18^17 + z18^16 + 3*z18^15 + 3*z18^14 + z18^13 + 2*z18^12 + 2*z18^11 + 2*z18^10 + 2*z18^9 + 4*z18^8 + 2*z18^7 + 3*z18^5 + 3*z18^4 + 4*z18 + 4]

In [81]:
s = l[0]; s

z18^17 + z18^16 + 3*z18^15 + 3*z18^14 + z18^13 + 2*z18^12 + 2*z18^11 + 2*z18^10 + 2*z18^9 + 4*z18^8 + 2*z18^7 + 3*z18^5 + 3*z18^4 + 4*z18 + 4

In [85]:
s^(p^6) == s

True

In [87]:
aa = T.random_element(); aa

(2*z9^8 + 2*z9^6 + z9^5 + z9^4 + z9^3 + 2*z9^2 + 1)*x^5 + (2*z9^8 + 4*z9^7 + 3*z9^5 + z9^4 + z9^3 + 2*z9)*x^4 + (3*z9^8 + 2*z9^6 + z9^5 + 4*z9^4 + 2*z9^3 + 2*z9^2 + 3*z9)*x^3 + (2*z9^7 + 2*z9^6 + 4*z9^4 + 3*z9^3 + 3*z9^2 + 3)*x^2 + (2*z9^8 + 2*z9^6 + z9^5 + 2*z9^4 + 4*z9^3 + 4*z9^2 + 4)*x + 2*z9^8 + 3*z9^7 + z9^6 + 4*z9^5 + 3*z9^4 + 3*z9^3 + z9 + 4

In [88]:
bb = riso(aa, K, C); bb

(2*z18^17 + 4*z18^16 + 4*z18^15 + 2*z18^13 + 4*z18^12 + 2*z18^11 + 4*z18^10 + z18^9 + 4*z18^8 + 2*z18^7 + 2*z18^6 + z18^5 + z18^4 + z18^2 + z18 + 1, z18^17 + 4*z18^16 + 2*z18^15 + z18^14 + 3*z18^13 + 4*z18^12 + 3*z18^11 + 2*z18^10 + z18^9 + 4*z18^8 + 3*z18^7 + 3*z18^6 + 2*z18^5 + 4*z18^4 + 3*z18^2 + 3*z18, 4*z18^17 + z18^15 + 2*z18^14 + 2*z18^13 + 2*z18^12 + 3*z18^11 + 4*z18^10 + 2*z18^8 + z18^7 + z18^6 + 2*z18^5 + 4*z18^4 + z18^3 + 3*z18^2 + 3*z18 + 2)

In [89]:
ll = [cc^n for cc in bb]; ll

[z18^17 + 3*z18^16 + 4*z18^15 + 3*z18^14 + z18^13 + 4*z18^10 + z18^8 + 4*z18^7 + 2*z18^6 + z18^5 + 3*z18^4 + 2*z18^3 + z18^2 + z18 + 2,
 2*z18^17 + 2*z18^16 + 4*z18^15 + z18^14 + z18^13 + 3*z18^12 + 2*z18^11 + 4*z18^10 + 3*z18^9 + 4*z18^8 + 4*z18^7 + 2*z18^6 + z18^5 + 2*z18^4 + 4*z18^2 + 2*z18,
 4*z18^17 + 2*z18^16 + z18^15 + 3*z18^14 + 4*z18^13 + z18^12 + 3*z18^9 + 2*z18^8 + 2*z18^6 + 3*z18^5 + 3*z18^4 + z18^3 + 2*z18^2 + 4]

In [90]:
l

[z18^17 + z18^16 + 3*z18^15 + 3*z18^14 + z18^13 + 2*z18^12 + 2*z18^11 + 2*z18^10 + 2*z18^9 + 4*z18^8 + 2*z18^7 + 3*z18^5 + 3*z18^4 + 4*z18 + 4,
 z18^17 + z18^16 + 3*z18^15 + 3*z18^14 + z18^13 + 2*z18^12 + 2*z18^11 + 2*z18^10 + 2*z18^9 + 4*z18^8 + 2*z18^7 + 3*z18^5 + 3*z18^4 + 4*z18 + 4,
 z18^17 + z18^16 + 3*z18^15 + 3*z18^14 + z18^13 + 2*z18^12 + 2*z18^11 + 2*z18^10 + 2*z18^9 + 4*z18^8 + 2*z18^7 + 3*z18^5 + 3*z18^4 + 4*z18 + 4]

In [107]:
(b[1]/b[0])^2 == (b[2]/b[0])

True

In [7]:
T

Univariate Quotient Polynomial Ring in x over Finite Field in z9 of size 5^9 with modulus x^6 + x^4 + 4*x^3 + x^2 + 2

In [57]:
p = 3
n = 4
r = Zmod(n)(p).multiplicative_order()
K = GF(p^n)
C = GF(p^r)
T = K.extension(C.modulus())
R.<x> = C[]
S.<y> = T[]

In [58]:
Z = y^n-1; Z

y^4 + 2

In [59]:
T.is_finite()

True

In [60]:
T.order(), log(T.order()*1., 2)

(6561, 12.6797000057692)

In [61]:
T.is_field()

False

In [63]:
cpt = 0
for t in T:
    if Z(t) == 0:
        cpt += 1
        print t

2*z4^3 + 2*z4^2 + 2
2
z4^3 + z4^2 + 1
1
(2*z4^3 + 2*z4^2 + 1)*x
(2*z4^3 + 2*z4^2 + 1)*x + z4^3 + z4^2 + 2
(2*z4^3 + 2*z4^2 + 2)*x + 2*z4^3 + 2*z4^2 + 2
(z4^3 + z4^2)*x
(z4^3 + z4^2)*x + 2*z4^3 + 2*z4^2
2*x + 2
(z4^3 + z4^2 + 2)*x
(z4^3 + z4^2 + 2)*x + 2*z4^3 + 2*z4^2 + 1
(z4^3 + z4^2 + 1)*x + z4^3 + z4^2 + 1
(2*z4^3 + 2*z4^2)*x
(2*z4^3 + 2*z4^2)*x + z4^3 + z4^2
x + 1


In [64]:
cpt

16

In [68]:
elem = riso(T.random_element(), K, C); elem

(2*z4 + 1, z4^3 + 2*z4 + 1)

In [69]:
elem.parent()

Vector space of dimension 2 over Finite Field in z4 of size 3^4

# Some other tests with norms
About constructing compatible roots of unity with respect to the relative norms

In [48]:
p = 19
l = 3
k = GF(p)
R = PolynomialRing(k, "x")

In [60]:
f2 = R(cyclotomic_polynomial(21)).factor(); f2
f1 = R(cyclotomic_polynomial(3)).factor(); f1

(x + 8) * (x + 12)

In [61]:
z = f1[0][0]; z

x + 8

In [62]:
u = f2[0][0]; u

x^6 + 7*x^5 + 11*x^4 + x^3 + 7*x^2 + 11*x + 1

In [63]:
k2.<z2> = k.extension(u)
k1.<z1> = k.extension(z)

In [56]:
z2.norm()

1

In [64]:
def relative_norm(x, k):
    K = x.parent()
    d = K.degree()//k.degree()
    q = k.cardinality()
    N = product([x^(q^j) for j in range(d)])
    return N

In [65]:
relative_norm(z2, k1)

1

In [37]:
z1.minpoly()

x^4 + 2*x^3 + x^2 + 2*x + 1

In [29]:
product(range(1,5))

24

In [42]:
p = 5
l = 13
r = Zmod(l^1)(p).multiplicative_order()


In [43]:
r

4

In [44]:
for p in primes_first_n(20)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71]

Case $p=19$, with $\zeta_3$ and $\zeta_{21}$